In [1]:
import requests
import pandas as pd

In [2]:
# http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,87.38&date=2008-09-08&tp=1&format=json
# above is example

In [3]:
#import cleaned NFL schedule
schedule_csv = "NFL_Schedule.csv"

schedule = pd.read_csv(schedule_csv)
schedule.head()

,Full Date,Time,time_conv,Week,City,Home Team,Winner
0,2008-09-04,7:08PM,1900,1,New York,Giants,Giants
1,2008-09-07,1:02PM,1300,1,Philadelphia,Eagles,Eagles
2,2008-09-07,1:02PM,1300,1,Miami,Dolphins,Jets
3,2008-09-07,1:02PM,1300,1,New England,Patriots,Patriots
4,2008-09-07,1:02PM,1300,1,Pittsburgh,Steelers,Steelers


In [4]:
#import cleaned NFL Stadium Locations
locations_csv = "NFL_Locations.csv"
locations = pd.read_csv(locations_csv)
locations

,Franchise,Latitude,Longitude
0,Chicago,41.53,-87.379997
1,Green Bay,44.30,-88.010002
2,Arizona,33.27,-112.050003
3,Baltimore,39.17,-76.370003
4,Buffalo,42.54,-78.529999
5,Carolina,35.14,-80.500000
6,Cincinnati,39.06,-84.309998
7,Cleveland,41.30,-81.410004
8,Denver,39.43,-105.010002
9,Indianapolis,39.46,-86.089996


In [5]:
locations["Latitude"] = locations["Latitude"].round(decimals=3)
locations["Longitude"] = locations["Longitude"].round(decimals=3)
locations

,Franchise,Latitude,Longitude
0,Chicago,41.53,-87.38
1,Green Bay,44.30,-88.01
2,Arizona,33.27,-112.05
3,Baltimore,39.17,-76.37
4,Buffalo,42.54,-78.53
5,Carolina,35.14,-80.50
6,Cincinnati,39.06,-84.31
7,Cleveland,41.30,-81.41
8,Denver,39.43,-105.01
9,Indianapolis,39.46,-86.09


In [7]:
locations.dtypes

Franchise     object
Latitude     float64
Longitude    float64
dtype: object

In [8]:
schedule.dtypes

Full Date    object
Time         object
time_conv     int64
Week         object
City         object
Home Team    object
Winner       object
dtype: object

In [10]:
#merge schedule with locations
merge = pd.merge(left=schedule, right=locations, how="left", left_on="City", right_on="Franchise")
merge.head()

,Full Date,Time,time_conv,Week,City,Home Team,Winner,Franchise,Latitude,Longitude
0,2008-09-04,7:08PM,1900,1,New York,Giants,Giants,New York,40.43,-74.01
1,2008-09-07,1:02PM,1300,1,Philadelphia,Eagles,Eagles,Philadelphia,39.57,-75.07
2,2008-09-07,1:02PM,1300,1,Miami,Dolphins,Jets,Miami,25.46,-80.12
3,2008-09-07,1:02PM,1300,1,New England,Patriots,Patriots,New England,42.21,-71.04
4,2008-09-07,1:02PM,1300,1,Pittsburgh,Steelers,Steelers,Pittsburgh,40.26,-80.00


In [ ]:
api_loop_table = merge[["Full Date","]]

In [ ]:
#http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,-87.38&date=2008-09-08&tp=1&format=json